<a href="https://colab.research.google.com/github/CarlosMendez1997Col/GEE-GeeMap-App/blob/main/Download_Images_GeeMap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Automated script to download images from a Google Earth Engine using GeeMap

## Import Libraries and Packages

In [1]:
!pip install geemap
!pip install earthengine-api

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 30.3 MB/s eta 0:00:00


In [2]:
import ee
import geemap
import ee
import webbrowser
import ipywidgets as widgets
import geopandas as gpd
from datetime import datetime

## Autentication in Google Colab and GEE


In [3]:
auth_url = ee.Authenticate(auth_mode='notebook')
webbrowser.open(auth_url)
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/cloud-platform%20https%3A//www.googleapis.com/auth/drive%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=FntnQ18zfHT1j7IsvVtwT9305GBZLsHz2KA-qm1NRT8&tc=RJjtFkrsdVSV-dIOXjRqD5I6YWkUfEoxbUP2AtORVo0&cc=En_SfwycJRaA9nYU3L5rM0JY7BL5mK6SHB4b5SEisUs

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1ASc3gC3d0vH30QdfvBRQWLlX_vUERTo1Se-bD8x4w0vDV8xn-UgABRI3ZXg

Successfully saved authorization token.


## Create basemaps and import Area Of Interest (AOI)

In [4]:
cobija_fc = ee.FeatureCollection("projects/gee-projects-481514/assets/CobijaSHP")

MapCobija = geemap.Map()
MapCobija.addLayer(cobija_fc, {"color": "darkblue", "opacity": 0.7}, "Cobija SHP")
MapCobija.centerObject(cobija_fc, 13)
MapCobija.setOptions('HYBRID')

styles = {
    "OpenStreetMap": "OpenStreetMap",
    "Esri World Street Map": "Esri.WorldStreetMap",
    "Esri World Imagery": "Esri.WorldImagery",
    "Esri World Topo Map": "Esri.WorldTopoMap",
    "Esri Ocean Basemap": "Esri.OceanBasemap",
    "Esri World Gray Canvas": "Esri.WorldGrayCanvas",
    "Esri World Physical": "Esri.WorldPhysical",
    "Esri World Shaded Relief": "Esri.WorldShadedRelief",
    "Esri World Terrain": "Esri.WorldTerrain",
    "CartoDB DarkMatter": "CartoDB.DarkMatter",
    "CartoDB Positron": "CartoDB.Positron",
    "CartoDB Voyager": "CartoDB.Voyager",
    "OpenTopoMap": "OpenTopoMap"
}

style_dropdown = widgets.Dropdown(
    options=list(styles.keys()),
    value="Esri World Imagery",
    description="Basemap:"
)

def update_style(change):
    MapCobija.add_basemap(styles[change['new']])
    MapCobija.addLayer(cobija_fc, {"color": "darkblue", "opacity": 0.7}, "Cobija SHP")
    MapCobija.centerObject(cobija_fc, 13)

style_dropdown.observe(update_style, names='value')

display(style_dropdown)
MapCobija

Dropdown(description='Basemap:', index=2, options=('OpenStreetMap', 'Esri World Street Map', 'Esri World Image…

Map(center=[-11.031617083854373, -68.77127014491582], controls=(WidgetControl(options=['position', 'transparen…

# Search new images in the AOI

## Landsat 9 OLI-2/TIRS-2

### Landsat 9 OLI-2/TIRS-2 Collection 2 atmospherically corrected surface reflectance.

In [6]:
cobija_fc = ee.FeatureCollection("projects/gee-projects-481514/assets/CobijaSHP")
landsat_collection = ee.ImageCollection("LANDSAT/LC09/C02/T1_L2").filterBounds(cobija_fc)
first_image = landsat_collection.sort('system:time_start').first()
start_date = ee.Date(first_image.get('system:time_start')).format('YYYY-MM-dd').getInfo()
end_date = datetime.utcnow().strftime("%Y-%m-%d")

landsat = (landsat_collection
           .filterDate(start_date, end_date)
           .filter(ee.Filter.lt('CLOUD_COVER', 20))
           .sort('system:time_start', False))

landsat_10 = landsat.limit(10)
images_list_10 = landsat_10.toList(landsat_10.size())

def apply_scale(image):
    scale = ee.Image(image).select(['SR_B.*']).multiply(0.0000275).add(-0.2)
    return image.addBands(scale, overwrite=True)

images_list = landsat.toList(landsat.size())
info_list = []
for i in range(landsat.size().getInfo()):
    img = ee.Image(images_list.get(i))
    info = img.getInfo()
    info_list.append({
        'ID': info['id'],
        'Nubosidad': info['properties']['CLOUD_COVER'],
        'Fecha inicio': info['properties']['system:time_start']
    })

print("Imágenes Landsat con baja nubosidad en Cobija:")
for info in info_list:
    print(info)

MapCobijaLandsat9SR = geemap.Map()

for i in range(landsat_10.size().getInfo()):
    img = ee.Image(images_list_10.get(i))
    img = apply_scale(img)  # aplicar escala a bandas SR
    date_str = ee.Date(img.get('system:time_start')).format('YYYY-MM-dd').getInfo()
    MapCobijaLandsat9SR.addLayer(img, {"bands":["SR_B4","SR_B3","SR_B2"], "min":0.05, "max":0.3}, f"Landsat {date_str} (nubes <20%)")

MapCobijaLandsat9SR.addLayer(cobija_fc, {"color":"darkblue", "opacity":0.7}, "Cobija SHP")
MapCobijaLandsat9SR.centerObject(cobija_fc, 14)
MapCobijaLandsat9SR

Imágenes Landsat con baja nubosidad en Cobija:
{'ID': 'LANDSAT/LC09/C02/T1_L2/LC09_002068_20250910', 'Nubosidad': 11.87, 'Fecha inicio': 1757515216229}
{'ID': 'LANDSAT/LC09/C02/T1_L2/LC09_002068_20250724', 'Nubosidad': 0.07, 'Fecha inicio': 1753367994974}
{'ID': 'LANDSAT/LC09/C02/T1_L2/LC09_002068_20250708', 'Nubosidad': 0, 'Fecha inicio': 1751985586375}
{'ID': 'LANDSAT/LC09/C02/T1_L2/LC09_002068_20250622', 'Nubosidad': 1.93, 'Fecha inicio': 1750603182575}
{'ID': 'LANDSAT/LC09/C02/T1_L2/LC09_002068_20250606', 'Nubosidad': 18.74, 'Fecha inicio': 1749220768398}
{'ID': 'LANDSAT/LC09/C02/T1_L2/LC09_002068_20241228', 'Nubosidad': 0.03, 'Fecha inicio': 1735396812912}
{'ID': 'LANDSAT/LC09/C02/T1_L2/LC09_002068_20241126', 'Nubosidad': 16.57, 'Fecha inicio': 1732632019312}
{'ID': 'LANDSAT/LC09/C02/T1_L2/LC09_002068_20240923', 'Nubosidad': 1.27, 'Fecha inicio': 1727102387862}
{'ID': 'LANDSAT/LC09/C02/T1_L2/LC09_002068_20240822', 'Nubosidad': 9.9, 'Fecha inicio': 1724337587530}
{'ID': 'LANDSAT/LC

Map(center=[-11.031617083854373, -68.77127014491582], controls=(WidgetControl(options=['position', 'transparen…

### Landsat 9 OLI-2/TIRS-2 Collection 2 DN values, representing scaled, calibrated at-sensor radiance.

In [7]:
cobija_bounds = cobija_fc.geometry().bounds()
landsat_collection = ee.ImageCollection("LANDSAT/LC09/C02/T1").filterBounds(cobija_fc)
first_image = landsat_collection.sort('system:time_start').first()
start_date = ee.Date(first_image.get('system:time_start')).format('YYYY-MM-dd').getInfo()
end_date = datetime.utcnow().strftime("%Y-%m-%d")

landsat = (landsat_collection
           .filterDate(start_date, end_date)
           .filter(ee.Filter.lt('CLOUD_COVER', 20))
           .sort('system:time_start', False))

images_list = landsat.toList(landsat.size())
info_list = []
for i in range(landsat.size().getInfo()):
    img = ee.Image(images_list.get(i))
    info = img.getInfo()
    info_list.append({
        'ID': info['id'],
        'Nubosidad': info['properties']['CLOUD_COVER'],
        'Fecha inicio': info['properties']['system:time_start']
    })

print("Imágenes Landsat Nivel 1 con baja nubosidad en Cobija:")
for info in info_list:
    print(info)

landsat_10 = landsat.limit(10)
images_list_10 = landsat_10.toList(landsat_10.size())

MapCobijaLC09 = geemap.Map()
for i in range(landsat_10.size().getInfo()):
    img = ee.Image(images_list_10.get(i))
    date_str = ee.Date(img.get('system:time_start')).format('YYYY-MM-dd').getInfo()
    MapCobijaLC09.addLayer(img, {"bands":["B4","B3","B2"], "min":0, "max":10000}, f"Landsat {date_str} (nubes <20%)")

MapCobijaLC09.addLayer(cobija_fc, {"color":"darkblue", "opacity":0.7}, "Cobija SHP")
MapCobijaLC09.centerObject(cobija_fc, 14)
MapCobijaLC09

Imágenes Landsat Nivel 1 con baja nubosidad en Cobija:
{'ID': 'LANDSAT/LC09/C02/T1/LC09_002068_20250910', 'Nubosidad': 11.87, 'Fecha inicio': 1757515216229}
{'ID': 'LANDSAT/LC09/C02/T1/LC09_002068_20250724', 'Nubosidad': 0.07, 'Fecha inicio': 1753367994974}
{'ID': 'LANDSAT/LC09/C02/T1/LC09_002068_20250708', 'Nubosidad': 0, 'Fecha inicio': 1751985586375}
{'ID': 'LANDSAT/LC09/C02/T1/LC09_002068_20250622', 'Nubosidad': 1.93, 'Fecha inicio': 1750603182575}
{'ID': 'LANDSAT/LC09/C02/T1/LC09_002068_20250606', 'Nubosidad': 18.74, 'Fecha inicio': 1749220768398}
{'ID': 'LANDSAT/LC09/C02/T1/LC09_002068_20241228', 'Nubosidad': 0.03, 'Fecha inicio': 1735396812912}
{'ID': 'LANDSAT/LC09/C02/T1/LC09_002068_20241126', 'Nubosidad': 16.57, 'Fecha inicio': 1732632019312}
{'ID': 'LANDSAT/LC09/C02/T1/LC09_002068_20240923', 'Nubosidad': 1.27, 'Fecha inicio': 1727102387862}
{'ID': 'LANDSAT/LC09/C02/T1/LC09_002068_20240822', 'Nubosidad': 9.9, 'Fecha inicio': 1724337587530}
{'ID': 'LANDSAT/LC09/C02/T1/LC09_0020

Map(center=[-11.031617083854373, -68.77127014491582], controls=(WidgetControl(options=['position', 'transparen…

## Sentinel

### Sentinel-1 SAR GRD: C-band Synthetic Aperture Radar Ground Range Detected

#### VV Polarization

#### VH Polarization

### Sentinel-2 MSI: MultiSpectral Instrument, Level-2A (SR)

